# Analyzing Used Car Listings on eBay Kleinanzeigen
We've made a few modifications from the original dataset:

- We sampled 50,000 data points from the full dataset, to ensure your code runs quickly in our hosted environment
- We dirtied the dataset a bit to more closely resemble what you would expect from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with)

#### The data dictionary provided with data is as follows:

- dateCrawled - When this ad was first crawled. All field-values are taken from this date.
- name - Name of the car.
- seller - Whether the seller is private or a dealer.
- offerType - The type of listing
- price - The price on the ad to sell the car.
- abtest - Whether the listing is included in an A/B test.
- vehicleType - The vehicle Type.
- yearOfRegistration - The year in which the car was first registered.
- gearbox - The transmission type.
- powerPS - The power of the car in PS.
- model - The car model name.
- kilometer - How many kilometers the car has driven.
- monthOfRegistration - The month in which the car was first registered.
- fuelType - What type of fuel the car uses.
- brand - The brand of the car.
- notRepairedDamage - If the car has a damage which is not yet repaired.
- dateCreated - The date on which the eBay listing was created.
- nrOfPictures - The number of pictures in the ad.
- postalCode - The postal code for the location of the vehicle.
- lastSeenOnline - When the crawler saw this ad last online.

#### The aim of this project is to clean the data and analyze the included used car listings. 

In [2]:
import pandas as pd
import numpy as np

In [3]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [4]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [5]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

- The dataset contains 20 columns, most of which are strings.
- Some columns have null values, but none have more than ~20% null values.
- The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

In [6]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

## Cleaning Column Names
Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [7]:
autos_columns_copy = autos.columns

mapping_dict = {
    'dateCrawled':'date_crawled',
    'name':'name',
    'seller':'seller',
    'offerType':'offer_type',
    'price':'price',
    'abtest':'abtest',
    'vehicleType':'vehicle_type',
    'yearOfRegistration':'registration_year',
    'gearbox':'gearbox',
    'powerPS':'power_ps',
    'model':'model',
    'odometer':'odometer',
    'monthOfRegistration':'registration_month',
    'fuelType':'fuel_type',
    'brand':'brand',
    'notRepairedDamage':'unrepaired_damage',
    'dateCreated':'ad_created',
    'nrOfPictures':'nr_of_pictures',
    'postalCode':'postal_code',
    'lastSeen':'last_seen'
}

autos.columns = pd.Series(autos_columns_copy).map(mapping_dict)
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## Initial Exploration and Cleaning

In [8]:
# (with include='all' to get both categorical and numeric columns) 
autos.describe(include="all") 

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-12 16:06:22,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


- Any columns that have mostly one value that are candidates to be dropped
- Any columns that need more investigation.
- Any examples of numeric data stored as text that needs to be cleaned.

Observation: 
- There are a number of text columns where all (or nearly all) of the values are the same:
 - seller
 - offer_type
- The num_photos column looks odd, we'll need to investigate this further.

So we will delete nr_of_pictures, offer_type, seller columns because they have mostly one value.


In [9]:
del autos['nr_of_pictures']
del autos['offer_type']
del autos['seller']


In [10]:
autos.head()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


Found that the `price` and `odometer`columns are numeric values stored as text. For each column:
- Remove any non-numeric characters.
- Convert the column to a numeric dtype.
- Use DataFrame.rename() to rename the column `odometer` to `odometer_km`.

In [17]:
autos['price'] = autos['price'].str.replace('$', '').str.replace(',', '').astype(int)
autos['odometer'] = autos['odometer'].str.replace('km','').str.replace(',','').astype(int)

autos.rename({'odometer':'odometer_km'}, axis=1, inplace=True)
autos.rename({'price':'price_usa_dollar'}, axis=1, inplace=True)


## Exploring the Odometer and Price Columns
#### Analyze the columns using minimum and maximum values and look for any values that look unrealistically high or low (outliers) that we might want to remove.


In [18]:
print('max:',max(autos['odometer_km']), 
      'min', min(autos['odometer_km']))

max: 150000 min 5000


In [24]:
# to see how many unique values
print("Odometer has", autos['odometer_km'].unique().shape[0], "unique values")
# view min/max/median/mean etc
autos['odometer_km'].describe()

Odometer has 13 unique values


count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [30]:
autos['odometer_km'].value_counts().sort_index(ascending=False)

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
40000       819
30000       789
20000       784
10000       264
5000        967
Name: odometer_km, dtype: int64

odometer statistics looks good, now let's see how the price is.

In [29]:
print('max:',max(autos['price_usa_dollar']), 
      'min', min(autos['price_usa_dollar']))

max: 99999999 min 0


We can easily recognize outliers (0 and 99999999 ?)<br>
Actually this data is scraped from eBay, the price can be any value but 0 and 99999999 seems not right. Let's continue exploring to find the answer.

In [28]:
# how many unique values
print("Price has", autos['price_usa_dollar'].unique().shape[0], "unique values")
# min/max/median/mean etc
# autos['price'].describe()
autos['price_usa_dollar'].value_counts().sort_index(ascending=True)


Price has 2357 unique values


0           1421
1            156
2              3
3              1
5              2
            ... 
10000000       1
11111111       2
12345678       3
27322222       1
99999999       1
Name: price_usa_dollar, Length: 2357, dtype: int64

### As we can see, price has many outliers.


As I search in google, [reference](https://www.beforward.jp/stocklist/fob_price_to=30), there is no used car that has price less than 30, so I will pick range from 30 -> 350000 to remove outlier beyond or less than its range

In [16]:
# removing row that has price greater than 350000 and less than 1

In [31]:
autos[autos['price_usa_dollar'].between(30, 350000)] 
# return a dataframe, can't assign a dataframe to series autos['price']

,date_crawled,name,price_usa_dollar,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,24900,control,limousine,2011,automatik,239,q5,100000,1,diesel,audi,nein,2016-03-27 00:00:00,82131,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,1980,control,cabrio,1996,manuell,75,astra,150000,5,benzin,opel,nein,2016-03-28 00:00:00,44807,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,Fiat_500_C_1.2_Dualogic_Lounge,13200,test,cabrio,2014,automatik,69,500,5000,11,benzin,fiat,nein,2016-04-02 00:00:00,73430,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,22900,control,kombi,2013,manuell,150,a3,40000,11,diesel,audi,nein,2016-03-08 00:00:00,35683,2016-04-05 16:45:07


there is 48370 rows that fit our range, let remove the other

In [49]:
autos = autos[autos['price_usa_dollar'].between(30, 350000)]

## Exploring the date columns

In [34]:
# three columns below are added and stored by string type, 
# we need to convert data into a numerical representation
# so we can understand it quantitatively.
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


### ad_created

In [44]:
pd.options.display.max_rows = 50

autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038846
2016-04-04    0.036882
2016-04-05    0.011784
2016-04-06    0.003246
2016-04-07    0.001240
Name: ad_created, Length: 76, dtype: float64

After 09-03-2016, it is began to have many ads. So I think before that, ads has been created and not updated anymore(I mean it is outlier)

In [46]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.025346
2016-03-06    0.014058
2016-03-07    0.036055
2016-03-08    0.033264
2016-03-09    0.033058
2016-03-10    0.032231
2016-03-11    0.032562
2016-03-12    0.036965
2016-03-13    0.015650
2016-03-14    0.036655
2016-03-15    0.034277
2016-03-16    0.029481
2016-03-17    0.031590
2016-03-18    0.012901
2016-03-19    0.034753
2016-03-20    0.037771
2016-03-21    0.037296
2016-03-22    0.032934
2016-03-23    0.032293
2016-03-24    0.029419
2016-03-25    0.031507
2016-03-26    0.032272
2016-03-27    0.031094
2016-03-28    0.034877
2016-03-29    0.034112
2016-03-30    0.033761
2016-03-31    0.031838
2016-04-01    0.033740
2016-04-02    0.035518
2016-04-03    0.038598
2016-04-04    0.036510
2016-04-05    0.013066
2016-04-06    0.003163
2016-04-07    0.001385
Name: date_crawled, dtype: float64

In [47]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.001075
2016-03-06    0.004321
2016-03-07    0.005417
2016-03-08    0.007360
2016-03-09    0.009634
2016-03-10    0.010626
2016-03-11    0.012384
2016-03-12    0.023775
2016-03-13    0.008869
2016-03-14    0.012632
2016-03-15    0.015878
2016-03-16    0.016456
2016-03-17    0.028075
2016-03-18    0.007319
2016-03-19    0.015816
2016-03-20    0.020612
2016-03-21    0.020591
2016-03-22    0.021398
2016-03-23    0.018607
2016-03-24    0.019764
2016-03-25    0.019165
2016-03-26    0.016787
2016-03-27    0.015588
2016-03-28    0.020839
2016-03-29    0.022307
2016-03-30    0.024747
2016-03-31    0.023858
2016-04-01    0.022865
2016-04-02    0.024871
2016-04-03    0.025181
2016-04-04    0.024519
2016-04-05    0.124995
2016-04-06    0.221584
2016-04-07    0.132086
Name: last_seen, dtype: float64

We can guess that this website works very well, since "last_seen" value is so small, we can imagine that cars have been sold so the uploader don't go to the website anymore. Or the website didn't help, so the customer didn't comeback.

In [48]:
autos['registration_year'].describe()

count    48370.000000
mean      2004.777300
std         88.817387
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

## Dealing with Incorrect Registration Year Data
We have 1000 and 9999 are outliers, because registration_year is the year that cars had been registered so it can't be 1000 and 9999.

In [50]:
autos['registration_year'].value_counts(normalize=True).sort_index()

1000    0.000021
1001    0.000021
1111    0.000021
1800    0.000041
1910    0.000062
          ...   
5911    0.000021
6200    0.000021
8888    0.000021
9000    0.000021
9999    0.000062
Name: registration_year, Length: 95, dtype: float64

As my knowledge after Google search, our range would be higher than 1930 and less than 2016


In [51]:
autos = autos[autos['registration_year'].between(1930, 2016)]

In [52]:
autos['registration_year'].value_counts(normalize=True).sort_index()

1931    0.000022
1934    0.000043
1937    0.000086
1938    0.000022
1939    0.000022
          ...   
2012    0.028157
2013    0.017230
2014    0.014240
2015    0.008303
2016    0.025963
Name: registration_year, Length: 75, dtype: float64

I think the time they use a car and sell it is 15 - 20 years
# Let's Exploring Price by Brand

In [54]:
brands = (autos["brand"].value_counts(normalize=True))
is_greater_than_1p = brands > 0.01
selected_brands = brands[is_greater_than_1p].index

In [56]:
brand_mean_prices_dict = {}
for brand in selected_brands:
    selected_brand = autos.loc[autos["brand"] == brand, :]
    brand_mean = selected_brand['price_usa_dollar'].mean()
    brand_mean_prices_dict[brand] = brand_mean

In [29]:
brand_mean_prices_dict

{'audi': 9371.45305514158,
 'bmw': 8369.67455505574,
 'citroen': 3784.924961715161,
 'fiat': 2818.456066945607,
 'ford': 3755.9139256071317,
 'hyundai': 5376.740899357602,
 'mazda': 4112.596614950635,
 'mercedes_benz': 8661.1348640214,
 'nissan': 4750.063202247191,
 'opel': 2995.3961507618283,
 'peugeot': 3100.6928057553955,
 'renault': 2484.749087591241,
 'seat': 4417.942285041225,
 'skoda': 6393.018348623853,
 'smart': 3591.0212443095597,
 'toyota': 5167.091062394604,
 'volkswagen': 5420.53840675552}

We aggregated across brands to understand mean price. We observed that in the top 6 brands, there's a distinct price gap.
- Audi, BWM, and Mercedes Benz
- Renault, Fiat are less expensive.
- Skoda, Volkswagen, Huyndai are beetween

While our natural instinct may be to display both aggregated series objects and visually compare them, this has a few limitations:
- it's difficult to compare more than two aggregate series objects if we want to extend to more columns
- we can't compare more than a few rows from each series object
- we can only sort by the index (brand name) of both series objects so we can easily make visual comparisons

Instead, we can combine the data from both series objects into a single dataframe (with a shared index) and display the dataframe directly. To do this, we'll need to use two pandas methods.
- pandas series constructor
- pandas dataframe constructor

In [57]:
# convert dictionary to series object
bmp_series = pd.Series(brand_mean_prices_dict)
print(bmp_series)

volkswagen       5420.538407
bmw              8369.674555
opel             2995.396151
mercedes_benz    8661.134864
audi             9371.453055
ford             3755.913926
renault          2484.749088
peugeot          3100.692806
fiat             2818.456067
seat             4417.942285
skoda            6393.018349
nissan           4750.063202
mazda            4112.596615
smart            3591.021244
citroen          3784.924962
toyota           5167.091062
hyundai          5376.740899
dtype: float64


In [58]:
# Get mean mileage of each brand
brand_mean_mileage_dict = {}
for brand in selected_brands:
    selected_brand = autos.loc[autos["brand"] == brand, :]
    brand_mean = selected_brand['odometer_km'].mean()
    brand_mean_mileage_dict[brand] = brand_mean

In [59]:
# convert dictionary to series object
mileage_series = pd.Series(brand_mean_mileage_dict)
print(mileage_series)

volkswagen       128738.935802
bmw              132628.593781
opel             129386.527666
mercedes_benz    130895.006687
audi             129259.811227
ford             124303.719643
renault          128200.273723
peugeot          127122.302158
fiat             117066.945607
seat             121166.077739
skoda            110884.665793
nissan           118370.786517
mazda            124464.033850
smart             99590.288316
citroen          119647.779479
toyota           115944.350759
hyundai          106541.755889
dtype: float64


In [61]:
# Create a dataframe from the first series object using the dataframe constructor.
df = pd.DataFrame(data=bmp_series, columns=['mean_prices'])
# assign next column into data frame
# df.assign(name of column, series name)
df = df.assign(mean_mileage=mileage_series)

In [62]:
df

,mean_prices,mean_mileage
volkswagen,5420.538407,128738.935802
bmw,8369.674555,132628.593781
opel,2995.396151,129386.527666
mercedes_benz,8661.134864,130895.006687
audi,9371.453055,129259.811227
ford,3755.913926,124303.719643
renault,2484.749088,128200.273723
peugeot,3100.692806,127122.302158
fiat,2818.456067,117066.945607
seat,4417.942285,121166.077739


We'll calculate ratio of mean_mileage / mean_prices to easy to analysing the agrregate data:

In [63]:
ratio = {}
for i in range(len(df)):
    ratio[df.index[i]] = round(df.iloc[i][1]/df.iloc[i][0], 2)

## Convert dict to series object to make it more easier to read:

In [65]:
# Note that the higher the value is, the lower the quality is.
ratio_series = pd.Series(ratio)
print(ratio_series)

volkswagen       23.75
bmw              15.85
opel             43.20
mercedes_benz    15.11
audi             13.79
ford             33.10
renault          51.59
peugeot          41.00
fiat             41.54
seat             27.43
skoda            17.34
nissan           24.92
mazda            30.26
smart            27.73
citroen          31.61
toyota           22.44
hyundai          19.82
dtype: float64


The range of car mileages does not vary as much as the prices do by brand, instead all falling within 10% for the top brands. <br>
There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.

# I will stop here for later comeback.

#### Data cleaning next steps:
- Identify categorical data that uses german words, translate them and map the values to their english counterparts
- Convert the dates to be uniform numeric data, so "2016-03-21" becomes the integer 20160321.
- See if there are particular keywords in the name column that you can extract as new columns.
<br>
#### Analysis next steps:

- Find the most common brand/model combinations
- Split the odometer_km into groups, and use aggregation to see if average prices follows any patterns based on the milage.
- How much cheaper are cars with damage than their non-damaged counterparts?